<a href="https://colab.research.google.com/github/andrewshawl/Vuelos/blob/main/Vuelos_AS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd
import requests 
import threading

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Load the data
tickets = pd.read_csv('/content/drive/MyDrive/challenge_dataset.csv',  parse_dates=True)

In [46]:
ciudades_dest = [x for x in tickets["destination"]]
ciudades_origen = [x for x in tickets["origin"]]
ciudades = ciudades_dest + ciudades_origen
ciudades = list( dict.fromkeys(ciudades) )




In [41]:
def temp_de_ciudades(ciudades_i, ciudades_latlon):
  for ciudad in ciudades_i:
    if ciudad in ciudades_origen:
      lat=tickets[tickets["origin"] == ciudad ]["origin_latitude"].iloc[0]
      lon= tickets[tickets["origin"] == ciudad ]["origin_longitude"].iloc[0] 
    else:
      lat = tickets[tickets["destination"] == ciudad ]["destination_latitude"].iloc[0]
      lon= tickets[tickets["destination"] == ciudad ]["destination_longitude"].iloc[0]
  
    
    url="http://api.openweathermap.org/data/2.5/forecast?" + \
      "lat=" + str(lat)+ "&lon=" + str(lon)+"&cnt=5" + \
      "&appid=" + "d1a4fab2d46987ad898458389baa88a2"

    try:
      response= requests.get(url)
    except Exception as e:
      print(f"Error getting temp for url:{url}.\n Skipping")


    weather_prev = response.json()

    # La API da la temperatura cada 3 horas, asi que el 5to elemento son 12 PM
    weather = weather_prev["list"][4]


    temp = weather["main"]["temp"]
    ciudades_latlon[ciudad] = (temp)








In [29]:
ciudades_latlon = {}
num_threads = 3
paso = len(ciudades) // num_threads
modulo = len(ciudades) % num_threads

# Si la division deja un residuo agregaos un thread extra
if modulo > 0:
  num_threads += 1
threads = [None] * num_threads


print(f"Haciendo las llamadas al api de openwather en {num_threads} procesos")



for i in range(num_threads):
    if i == num_threads - 1 and modulo > 0:
      ciudades_i = ciudades[i*paso:(i*paso) + modulo]
    else:
      ciudades_i = ciudades[i*paso:(i+1)*paso]

    
    # print(ciudades_temp)
    threads[i] = threading.Thread(target=temp_de_ciudades,
                                  args=(ciudades_i,
                                        ciudades_latlon))
    threads[i].start()

for i in range(num_threads):
    threads[i].join()


ciudades_latlon


Haciendo las llamadas al api de openwather en 4 procesos


{'TLC': 290.97,
 'JFK': 301.3,
 'SCL': 294.39,
 'MTY': 307.59,
 'MZT': 304.67,
 'MEX': 295.9,
 'ACA': 300.01,
 'GUA': 297.6,
 'ZIH': 303.4,
 'TAM': 304.54,
 'AGU': 298.29,
 'GDL': 298.17,
 'CJS': 300.81,
 'VSA': 308.1,
 'BZE': 303.15,
 'CUN': 304.4,
 'CZM': 302.42,
 'DFW': 298.87,
 'CUU': 296.29,
 'TIJ': 300.6,
 'ORD': 300.75,
 'TRC': 304.86,
 'PHX': 308.34,
 'HMO': 304.98,
 'QRO': 297.63,
 'CME': 302.45,
 'PHL': 303.24,
 'BJX': 297.51,
 'MID': 309.43,
 'PBC': 294.38,
 'CLT': 304.52,
 'CTM': 303.8,
 'SLP': 298.69,
 'YYZ': 300.58,
 'VER': 303.51,
 'ZCL': 297.97,
 'OAX': 298.52,
 'LIM': 291.23,
 'HUX': 304.82,
 'IAH': 299.41,
 'HAV': 306.32,
 'PVR': 303.1,
 'YVR': 291.91,
 'BOG': 289.83,
 'CDG': 299.42,
 'MIA': 305.45,
 'AMS': 295.65,
 'PXM': 303.92,
 'ATL': 303.4,
 'LAX': 298.12,
 'CEN': 305.28,
 'MAD': 308.94}

In [47]:
tickets_reduced= tickets[["origin","destination","flight_num"]]
tickets_reduced=tickets_reduced.drop_duplicates(subset="flight_num", keep= "last")
temp_origen = [ciudades_latlon[x] for x in tickets_reduced["origin"] ]
temp_destiny=[ciudades_latlon[x] for x in tickets_reduced["destination"] ]

tickets_reduced["origin_weather"]=temp_origen
tickets_reduced['destination_weather']=temp_destiny 
tickets_reduced

,origin,destination,flight_num,origin_weather,destination_weather
1,MTY,TLC,119,307.59,290.97
2,MEX,MTY,2100,295.90,307.59
3,MTY,MEX,2101,307.59,295.90
9,MEX,MTY,2102,295.90,307.59
10,MTY,MEX,2103,307.59,295.90
...,...,...,...,...,...
2915,GDL,TIJ,110,298.17,300.60
2916,MEX,GDL,1104,295.90,298.17
2918,MEX,GDL,1120,295.90,298.17
2919,GDL,MEX,1127,298.17,295.90
